# Tareas
1) Desde un notebook, script para llamar a ecobici y guardar alguna info SIMPLE (ej: cuántos slots ocupados) en csv.
2) Agregar el secret para usar en github actions
3) Configurar la Github Action para que haga un commit (en una branch en particular? O en main?)
4) Luego de probar que funciona, agregar el cron a la github action
5) Mergear a main el nuevo código

In [1]:
import pandas as pd

from internal.platform.environment.environment import set_environment
from internal.gateway.transportBA import transportBA
from internal.repository.transportBA.stations_repository import StationsRepository
from internal.gateway.openrouteservice.client import OpenRouteServiceClient
from internal.gateway.openrouteservice.dataframe_adapter import OpenRouteServiceDataFrameAdapter

In [2]:
set_environment()

In [3]:
def preprocessStations(stations_df):
    return stations_df.explode("groups")

def preprocessStationsStatus(stations_status_df):
    stations_status_df["status_last_reported_utc"] = pd.to_datetime(stations_status_df["last_reported"], unit="s", utc=True)
    return station_status_df

In [4]:
# update stations.csv
transportBAClient = transportBA.Client()
stations_df = transportBAClient.GetStations()
station_status_df = transportBAClient.GetStationStatus()
stationsDao = StationsRepository()

stations_df = preprocessStations(stations_df)
station_status_df = preprocessStationsStatus(station_status_df)
stations_with_status_df = pd.merge(stations_df, station_status_df, on="station_id")

stationsDao.AppendWithJobRunAt(stations_with_status_df)

In [5]:
from internal.repository.transportBA.journeys_repository import JourneysRepository
journeysDao = JourneysRepository()
dataFrameAdapterClient = OpenRouteServiceDataFrameAdapter(client=OpenRouteServiceClient())

length = len(stations_df)
lower, stations_per_req = 0, 59  # max of 59 elements in request
journeys_df = pd.DataFrame()
for i in range(0, int(length/59)):
    stations_df_short = stations_df.iloc[lower*stations_per_req:(lower+1)*stations_per_req]
    df = dataFrameAdapterClient.GetDurationMatrixFromDataFrame("cycling-regular", stations_df_short)
    journeys_df = pd.concat([journeys_df, df], ignore_index=True)

journeysDao.Update(journeys_df, subset=["station_id_from", "station_id_to"])

{'locations': [[-58.37470988999999, -34.59242413], [-58.369129063788996, -34.61220714255728], [-58.36885646243477, -34.6030082348981], [-58.4209542, -34.5805497], [-58.369758, -34.628526], [-58.381098, -34.606498], [-58.3893364, -34.6094218], [-58.407740999999994, -34.585443], [-58.388807, -34.5927096], [-58.406, -34.61009], [-58.426387, -34.577424], [-58.4187306, -34.6064101], [-58.406432, -34.640111], [-58.3825498, -34.5938629], [-58.38089430000001, -34.610583], [-58.416117799999995, -34.5894269], [-58.3638723, -34.600752], [-58.3900887, -34.599068], [-58.4335573, -34.6079414], [-58.37365550287733, -34.60674458045258], [-58.398980699999996, -34.5970909], [-58.371847, -34.5964246], [-58.374534600000004, -34.6010599], [-58.382840300000005, -34.5970497], [-58.40588830000001, -34.6371232], [-58.38992100000001, -34.584018], [-58.4138829, -34.5755148], [-58.386760190213025, -34.59944049406817], [-58.37436969999998, -34.6163729], [-58.3632448, -34.6236397], [-58.42481977731553, -34.62917608